In [31]:
from pathlib import Path
import pydicom
import shutil
import os

In [11]:
trondheim_dicoms_dirs = list(Path('image_data/Eva_Mona/').glob('*/DICOM'))
st_olav_output = Path('image_data/st_olav_output_1509')

In [26]:
def get_subject_id_from_path(path):
    subject_part = path.parts[-2]
    subject_id = subject_part.split(' ')[0]
    modality = subject_part.split(' ')[1]
    return (subject_id, modality)

In [9]:
def check_if_file_is_dicom(path_to_file):
    try:
        dcm = pydicom.dcmread(path_to_file, stop_before_pixels=True, force=True)
        if isinstance(dcm, pydicom.dicomdir.DicomDir):
            return False
        elif isinstance(dcm, pydicom.dataset.FileDataset):
            return True
    except:
        return False

In [21]:
def scan_directory_for_files(path_to_dir):
    files = []
    for file in path_to_dir.glob('**/*'):
        if file.is_file():
            if check_if_file_is_dicom(file):
                files.append(file)
    return files

In [39]:
def convert_patient_id():
    for path in trondheim_dicoms_dirs:
        subject_id = get_subject_id_from_path(path)[0]
        modality = get_subject_id_from_path(path)[1]
        files = scan_directory_for_files(path)
        out_dir = st_olav_output / subject_id / modality
        os.makedirs(out_dir, exist_ok=True)
        for file in files:
            if check_if_file_is_dicom(file):
                dcm = pydicom.dcmread(file, stop_before_pixels=True, force=True)
                dcm.PatientID = subject_id
                dcm.PatientName = subject_id
                dcm.save_as(out_dir / file.name)